`참고: 덕성여대 Nano-degree 교과 <언어 빅데이터 이해와 활용>,<언어 빅데이터>`

```
작성자: 박민준
참고문헌:
Bird, S., Klein, E., & Loper, E. (2009). Natural language processing with Python: analyzing text with the natural language toolkit. " O'Reilly Media, Inc.".
이기창. 한국어 임베딩. 에이콘.
```

## 환경 설정

In [1]:
# 내 구글 드라이브에 연동하기 (주의:이 컴퓨터의 다른 사용자가 접근할 수 있음)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 루트 디렉토리 설정 (개인마다 다르겠죠?)
root_path='/content/drive/MyDrive/Classroom/빅데이터시스템-01'

# Word2Vec 실습코드

## 네이버 댓글 말뭉치로 임베딩 만들기

In [3]:
# 데이터 로드
from nltk.corpus.reader import TaggedCorpusReader
reader = TaggedCorpusReader(root_path + r'/data/tokenized', 'ratings_mecab.txt')

In [4]:
# 제대로 로드 되었는지 확인
reader.fileids()

['ratings_mecab.txt']

In [5]:
# word2vec 모델 로드
from gensim.models import Word2Vec

In [ ]:
model_fname = root_path + r'/model/ratings_mecab_noun.model' #1 네이버 댓글 명사 말뭉치

In [6]:
model_fname = root_path + r'/model/word2vec' #2 네이버 댓글 말뭉치

In [12]:
%%time
# param corpus: [[sentence1],[sentence2],... ]
corpus = list(reader.sents())
# corpus = [sent.strip().split(" ")for sent in open(corpus_fname,"r").readlines()]

# 모델 학습 후 저장
print("make word embeddings")
model = Word2Vec(corpus, vector_size=100, workers=4, sg=1)
model.save(model_fname)         # 행렬 model이 저장됨

make word embeddings
CPU times: user 2min 43s, sys: 1.17 s, total: 2min 45s
Wall time: 1min 57s


In [8]:
print(corpus[0])

['어릴', '때', '보', '고', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ']


In [9]:
from gensim.models import KeyedVectors

# 모델 로드
model = Word2Vec.load(model_fname)

# 가장 비슷한 단어 5개 출력
model.wv.most_similar("희망", topn=5)

[('깨달음', 0.7531225681304932),
 ('젊음', 0.7511354684829712),
 ('외로움', 0.7406880855560303),
 ('구원', 0.726189911365509),
 ('절망', 0.7247464656829834)]

In [13]:
# vocab size |V|
len(model.wv.index_to_key)

18790

In [15]:
# 앞에서 10번째까지의 어휘(vocab)들
model.wv.index_to_key[:10]

['.', '이', '는', '영화', '다', '고', '하', '도', '의', '가']

In [14]:
# word embedding size (V x N)
model.wv.vectors.shape

(18790, 100)

In [16]:
model.wv.vectors[0,:]       # 첫번째 단어의 벡터

array([-0.22619545,  0.22610833,  0.09921899,  0.01905351, -0.05284338,
       -0.02920034,  0.30169433,  0.39938006, -0.11468904, -0.18523955,
       -0.18466954, -0.783112  , -0.06325828,  0.50168455,  0.1417841 ,
       -0.07993726,  0.22312574, -0.04500309, -0.22140317, -0.24286632,
        0.31266943, -0.00812238,  0.41347846, -0.20372988,  0.10115337,
       -0.1302372 , -0.02858037, -0.02663953, -0.53715736, -0.0803109 ,
        0.37626344,  0.20016652, -0.03219735, -0.38852784, -0.01829882,
        0.22067878, -0.0431059 , -0.07724696, -0.23480575, -0.28338227,
        0.22958411,  0.0675462 , -0.31642532, -0.06383581,  0.14150976,
        0.05985986, -0.14564504,  0.01736852,  0.11975449,  0.13711117,
       -0.14026283, -0.02521649, -0.09038403,  0.07948575, -0.05344441,
       -0.12109989,  0.10096221, -0.01598394, -0.3410803 ,  0.19906043,
        0.21635836, -0.10281841,  0.1712403 ,  0.13451555,  0.00682355,
        0.32794154,  0.24205977,  0.18233173, -0.331324  ,  0.22

In [17]:
# WE의 첫 번째 행 벡터는 '.' word vector임.
model.wv.vectors[0,:] == model.wv['.']

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

### 개별 단어 임베딩 살펴보기

In [18]:
a = model.wv.vectors[0,:]
b = model.wv['.']

In [19]:
from numpy import dot
from numpy.linalg import norm
# cosine 유사도 대수 계산
dot(a,b)/(norm(a)*norm(b))

1.0000001

In [20]:
# cosine_similarity 함수도 있음.
# 똑같기 때문에 θ=0 cosθ=1
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([a],[b])

array([[1.0000001]], dtype=float32)

In [ ]:
import pandas as pd
wordlist = model.wv.index_to_key[:10]
similarTB = pd.DataFrame()

for wd in model.wv.index_to_key[:10]:
  similarTB[wd] = model.wv.most_similar(wd, topn=5)


similarTB

,영화,연기,최고,평점,스토리,생각,드라마,사람,감동,배우
0,"(영활, 0.8238646984100342)","(연기력, 0.8691237568855286)","(두말, 0.7508763670921326)","(댓글, 0.7991413474082947)","(내용, 0.7767152786254883)","(그건, 0.802037239074707)","(막장, 0.7934067249298096)","(반응, 0.7267737984657288)","(콧물, 0.7574648857116699)","(연기자, 0.7871526479721069)"
1,"(류영화, 0.8131572008132935)","(조연, 0.8039903044700623)","(연애시대, 0.7300649881362915)","(별점, 0.788658082485199)","(속도, 0.7623175382614136)","(여럿, 0.7971391677856445)","(시트콤, 0.7682916522026062)","(여럿, 0.7256540060043335)","(방울, 0.7491053938865662)","(양동근, 0.7867127656936646)"
2,"(영환, 0.8115177154541016)","(발연기, 0.799642026424408)","(과언, 0.7259496450424194)","(점수, 0.7801305651664734)","(줄거리, 0.7612058520317078)","(걸까, 0.7970914244651794)","(종영, 0.7657601833343506)","(타인, 0.7242242693901062)","(물결, 0.7429309487342834)","(안성기, 0.7762317657470703)"
3,"(악평, 0.8034228086471558)","(열연, 0.7681509256362915)","(역대, 0.7249169945716858)","(위원회, 0.7717190384864807)","(금포, 0.7551357746124268)","(착각, 0.7931740880012512)","(아침, 0.7581095695495605)","(주관, 0.7223556637763977)","(남녀노소, 0.7429003119468689)","(엑스트라, 0.7713273167610168)"
4,"(영화광, 0.7947927713394165)","(연기파, 0.764315664768219)","(자부, 0.7216507792472839)","(평정, 0.7710609436035156)","(조잡, 0.7522072792053223)","(일지, 0.7920054793357849)","(사극, 0.7508510947227478)","(주위, 0.7215460538864136)","(발레, 0.7340382933616638)","(소지섭, 0.769092321395874)"


## 다음 중 의미가 가장 다른 하나는?

In [ ]:
model.wv.doesnt_match('교수 코치 선생 호랑이'.split())

'선생'

In [ ]:
model.wv.similarity('선생', '호랑이')

0.4233041

In [ ]:
model.wv.similarity('코치', '호랑이')

0.686099

In [ ]:
# specific word vector
model.wv['선생'].shape

(100,)

In [ ]:
model.wv.most_similar("는", topn=5)

[('스페인어', 0.6708201766014099),
 ('따질', 0.6594117283821106),
 ('하곤', 0.6585707664489746),
 ('이를', 0.652357816696167),
 ('쉴드', 0.6491264700889587)]

In [ ]:
model.wv.most_similar("선생", topn=5)

[('목사', 0.7198830842971802),
 ('추기경', 0.7136739492416382),
 ('담임', 0.7121151685714722),
 ('께서', 0.7088024616241455),
 ('님', 0.7044419646263123)]

In [ ]:
model.wv.most_similar("코치", topn=5)

[('건방진', 0.8792516589164734),
 ('카를로', 0.8770885467529297),
 ('자건', 0.8768724799156189),
 ('흥신소', 0.8735184073448181),
 ('애타', 0.8735085129737854)]

In [ ]:
model.wv.most_similar("호랑이", topn=5)

[('계곡', 0.8236473798751831),
 ('캐스퍼', 0.8231244087219238),
 ('하치', 0.8156342506408691),
 ('마사미', 0.8132513761520386),
 ('엘리자베스', 0.8118702173233032)]

In [ ]:
model.wv.doesnt_match('남자 여자 한국'.split())

'한국'

## 워드 임베딩의 자질(feature vectors)

In [ ]:
model.wv.most_similar(positive=['평점'], negative=['최고'])

[('평', 0.5829541683197021),
 ('별점', 0.5062258243560791),
 ('댓글', 0.5010291337966919),
 ('리뷰', 0.42783868312835693),
 ('네이버', 0.41871875524520874),
 ('낮', 0.4181133210659027),
 ('악평', 0.4165099859237671),
 ('점수', 0.3910801112651825),
 ('평균', 0.3867323100566864),
 ('조작', 0.3813304603099823)]

In [ ]:
model.wv.most_similar(positive=['영화'], negative=['최고'])

[('공포물', 0.4448525607585907),
 ('영상물', 0.4212844669818878),
 ('영활', 0.3908607065677643),
 ('관객', 0.3707869350910187),
 ('설정', 0.3588887155056),
 ('떡밥', 0.3515338599681854),
 ('겉', 0.35084789991378784),
 ('대충', 0.3504796624183655),
 ('허세', 0.3490891456604004),
 ('것', 0.3484945297241211)]

In [ ]:
model.wv.most_similar(positive=['영화'], negative=['생각'])

[('공포물', 0.3992606997489929),
 ('무비', 0.38698315620422363),
 ('싸구려', 0.36874836683273315),
 ('저질', 0.3666098117828369),
 ('졸작', 0.35978880524635315),
 ('드라마', 0.3590864837169647),
 ('느와르', 0.35882532596588135),
 ('sf', 0.3555689752101898),
 ('스릴러물', 0.3552993834018707),
 ('헐리우드', 0.35191190242767334)]

In [ ]:
model.wv.most_similar(positive=['배우'], negative=['연기력'])

[('애', 0.37082886695861816),
 ('사람', 0.3469926118850708),
 ('관객', 0.3120112717151642),
 ('바보', 0.29315951466560364),
 ('아이', 0.29031461477279663),
 ('그룹', 0.2859272360801697),
 ('달', 0.2849668264389038),
 ('영상물', 0.284572571516037),
 ('맘대로', 0.2822958827018738),
 ('아줌마', 0.28025200963020325)]

In [ ]:
model.wv.most_similar(positive=['한국', '톱스타'], negative=['서양'])

[('카메오', 0.6429650187492371),
 ('당근', 0.6269338130950928),
 ('로버트드니로', 0.6234715580940247),
 ('안성기', 0.6193339824676514),
 ('브루스윌리스', 0.6180586814880371),
 ('송은채', 0.615935742855072),
 ('엑스트라', 0.612000584602356),
 ('김인권', 0.6086661219596863),
 ('에릭로버츠', 0.6082873940467834),
 ('Cg', 0.6075512170791626)]

## 한국어 위키피디아 말뭉치로 임베딩 만들기

In [ ]:
# 데이터 로드
from nltk.corpus.reader import TaggedCorpusReader
reader = TaggedCorpusReader(root_path + r'/data/processed', 'wiki_ko_mecab_noun.txt')

In [ ]:
# 제대로 로드 되었는지 확인
reader.fileids()

['wiki_ko_mecab_noun.txt']

In [ ]:
%%time
# word2vec 모델 로드
from gensim.models import Word2Vec

# param corpus: [[sentence1],[sentence2],... ]
# corpus = list(reader.sents())
# print("make Wiki_ko_corpus")

# 모델 학습 후 저장 1h 25min 52s
model = Word2Vec(list(reader.sents()), vector_size=100, workers=16, sg=1)
model_fname = root_path + r'/model/wiki_ko_mecab_noun_w2v.model'
model.save(model_fname)

CPU times: user 1h 25min 31s, sys: 20.2 s, total: 1h 25min 52s
Wall time: 51min 19s


In [ ]:
print(corpus[0])

['디자인', '학생', '외국', '디자이너', '전통', '발전', '문화', '산업', '우리', '나라', '시절', '열정', '노라노', '전통', '사람', '감사']


In [ ]:
from gensim.models import KeyedVectors

# 모델 로드
model = Word2Vec.load(model_fname)


# 가장 비슷한 단어 5개 출력
model.wv.most_similar("희망", topn=5)

[('행복', 0.7876692414283752),
 ('웃사랑', 0.7155756950378418),
 ('공감', 0.7133263349533081),
 ('홀씨', 0.7104679346084595),
 ('기아대책', 0.7087770700454712)]

In [ ]:
model.wv.index_to_key[:10]

['분류', '선수', '대한민국', '미국', '영화', '일본', '축구', '사람', '사용', '학교']

In [ ]:
import pandas as pd
wordlist = model.wv.index_to_key[:10]
similarTB = pd.DataFrame()

for wd in model.wv.index_to_key[:10]:
  similarTB[wd] = model.wv.most_similar(wd, topn=5)


similarTB

,분류,선수,대한민국,미국,영화,일본,축구,사람,사용,학교
0,"(백세인, 0.6476739048957825)","(축구, 0.8049657344818115)","(월남자, 0.6622364521026611)","(재널, 0.7794177532196045)","(스릴러, 0.8483357429504395)","(모토후미, 0.7882861495018005)","(선수, 0.8049657344818115)","(죽음, 0.726020872592926)","(용도, 0.8588557839393616)","(초등, 0.8764864802360535)"
1,"(가톨릭교도, 0.6388962268829346)","(공격수, 0.7453395128250122)","(벅와일즈, 0.6563129425048828)","(새디악, 0.7784264087677002)","(카사베티스, 0.812800943851471)","(기요미야, 0.7733545303344727)","(공격수, 0.776968240737915)","(애꾸눈이, 0.694724440574646)","(이용, 0.8212010264396667)","(중학교, 0.7805733680725098)"
2,"(호프스트라, 0.6384819746017456)","(수비수, 0.7396318316459656)","(예창근, 0.639543354511261)","(매런, 0.7773605585098267)","(크쥐시토프, 0.8117460012435913)","(후쿠조, 0.7614952921867371)","(수비수, 0.7732674479484558)","(화이트칼라, 0.655973494052887)","(유용, 0.7817366719245911)","(초등학교, 0.7573122978210449)"
3,"(인체냉동보존, 0.6282073855400085)","(이충성, 0.7137298583984375)","(女士, 0.6373187899589539)","(머윈, 0.7756816744804382)","(소넌펠드, 0.8102418780326843)","(에비누마, 0.7523475885391235)","(미드필더, 0.7609261274337769)","(여자, 0.6534224152565002)","(잉크통, 0.7768834233283997)","(분교, 0.7518923878669739)"
4,"(애꾸눈이, 0.6248880624771118)","(골키퍼, 0.7117104530334473)","(전도봉, 0.6313393115997314)","(캐머, 0.7693083882331848)","(아빌드센, 0.8078650832176208)","(쓰카다, 0.7492590546607971)","(골키퍼, 0.7440699934959412)","(개신교도, 0.6362790465354919)","(빵판, 0.7632306814193726)","(졸업, 0.7463243007659912)"


# FastText실습코드

#### 코드 4-8 https://github.com/ratsgo/embedding 사용

In [ ]:
!git clone https://github.com/ratsgo/embedding.git

In [ ]:
!pip install fasttext

In [ ]:
!pip install soynlp

In [ ]:
# 설치하려면 다운로드 해서 빌드해야 함
# 귀찮으니 아래 코드 4-8-1로 대체
!pip install khaiii

ERROR: Could not find a version that satisfies the requirement khaiii (from versions: none)
ERROR: No matching distribution found for khaiii


In [ ]:
%cd embedding

/content/embedding


In [ ]:
from models.word_eval import WordEmbeddingEvaluator

#### 코드 4-8-1 https://joyhong.tistory.com/135

In [ ]:
from gensim.models import FastText

model_fname =root_path + '/model/fasttext'
corpus_fname = root_path + '/data/processed/ratings_mecab_noun.txt'


# corpus 없다면 단어별로 나누어 생성
# corpus = [sent.strip().split(" ")for sent in open(corpus_fname,"r").readlines()]
corpus = list(reader.sents())

# 모델 학습
print("학습 중")
# Gensim's FastText implementation does not yet support word_ngrams != 1.
model = FastText(corpus, vector_size=100, workers=4, sg=1, word_ngrams=1)

model.save(model_fname)


학습 중


In [ ]:
from gensim.models import FastText
model = FastText.load(model_fname)
print(model.wv.most_similar("학교", topn=5))
print(model.wv.most_similar("초등헉교", topn=5))




[('초등', 0.8974595665931702), ('학년', 0.8800656199455261), ('대학교', 0.873077392578125), ('저학년', 0.8470569849014282), ('고등학교', 0.8421161770820618)]
[('초등', 0.9838671088218689), ('초등학생', 0.9626556634902954), ('저학년', 0.9559977054595947), ('학년', 0.9438982605934143), ('유치원', 0.9334425926208496)]


In [ ]:
import pandas as pd
similarTB = pd.DataFrame()

for wd in model.wv.index_to_key[:10]:
  similarTB[wd] = model.wv.most_similar(wd, topn=5)

similarTB



영화
연기
최고
평점
스토리
생각
드라마
사람
감동
배우


,영화,연기,최고,평점,스토리,생각,드라마,사람,감동,배우
0,"(류영화, 0.8589946627616882)","(연기력, 0.8921881318092346)","(과언, 0.7476285099983215)","(별점, 0.8193385601043701)","(스토, 0.8958631753921509)","(그건, 0.8041960597038269)","(드라만, 0.8436130285263062)","(여럿, 0.7086215019226074)","(물결, 0.7476613521575928)","(남배우, 0.8169156908988953)"
1,"(영화광, 0.8569265007972717)","(연기파, 0.8622297644615173)","(최고봉, 0.737854540348053)","(점수, 0.8144073486328125)","(토리, 0.8308815956115723)","(별생각, 0.8022514581680298)","(수목드라마, 0.8346712589263916)","(산사람, 0.7053708434104919)","(콧물, 0.7440598607063293)","(명배우, 0.7847272753715515)"
2,"(국영화, 0.8383467197418213)","(발연기, 0.8541439175605774)","(두말, 0.7284854054450989)","(리뷰, 0.8006409406661987)","(스토린, 0.8183914422988892)","(일지, 0.791238009929657)","(드라, 0.8247290849685669)","(작정, 0.7045159935951233)","(감동이, 0.7418437600135803)","(연기자, 0.7777551412582397)"
3,"(영화당, 0.8349733352661133)","(명연기, 0.8041539192199707)","(최악, 0.7259068489074707)","(평균, 0.7905896902084351)","(스토리텔링, 0.8127031326293945)","(보장, 0.7871499061584473)","(멜로드라마, 0.8025949001312256)","(제정신, 0.7009191513061523)","(신파극, 0.7403324842453003)","(여배우, 0.771973192691803)"
4,"(기영화, 0.8202558755874634)","(조연, 0.790408730506897)","(넘버원, 0.7245931029319763)","(최소, 0.7832680344581604)","(내용, 0.8077704906463623)","(인지, 0.7864639163017273)","(대하드라마, 0.7954343557357788)","(멍청이, 0.6965394616127014)","(방울, 0.7384477853775024)","(엑스트라, 0.7675785422325134)"


# Glove


In [ ]:
!pip install glove-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.0/327.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for glove-python3: filename=glove_python3-0.1.0-cp310-cp310-linux_x86_64.whl size=1065514 sha256=7901f4eb5a9d047778151e4733679056664bb7a48867389a70ded44b5a086edd
  Stored in directory: /root/.cache/pip/wheels/fe/2f/79/34314d44a0907e90e323c8c182ec23f126eb460829e02d98cf
Successfully built glove-python3


In [ ]:
from glove import Corpus, Glove
model_fname =root_path + '/model/glove'
corpus_glove = Corpus()
corpus_glove.fit(corpus, window=15)
# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성

glove = Glove(no_components=100, learning_rate=0.05)#?
glove.fit(corpus_glove.matrix, epochs=15, no_threads=4, verbose=True)
glove.add_dictionary(corpus_glove.dictionary)
# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 15
glove.save(model_fname)

Performing 15 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14


In [ ]:
# 모델 저장 후 load
from glove import Glove
glove = Glove.load(model_fname)
print(glove.most_similar('희망', number=10))


[('고통', 0.974308325813348), ('아픔', 0.9680312028898697), ('절망', 0.9678999015948601), ('진실', 0.9644287836684461), ('현재', 0.9624063337554133), ('가정', 0.9581420516595902), ('죽음', 0.9576264973060807), ('기적', 0.9575511513403661), ('누군가', 0.9554043551016562)]


In [ ]:
similarTB = pd.DataFrame()

for wd in wordlist:
  similarTB[wd] = glove.most_similar(wd, number=6)

similarTB

,무엇,이름,사람,누구,미국,이후,당시,일본,시작,때문
0,"(가지, 0.9649784575255584)","(본인, 0.9611563244717545)","(입장, 0.9424177297908174)","(그것, 0.9747720402566816)","(문화, 0.9666553359724221)","(내인생의, 0.9211642020540365)","(개봉, 0.9556435249748022)","(한국, 0.9626575680001384)","(복수, 0.9494550444966383)","(텐데, 0.9610461071216052)"
1,"(그것, 0.9616729857551282)","(서극, 0.9496367959745667)","(관심, 0.9423357472587399)","(얘기, 0.9616297649008607)","(중국, 0.9562188304067482)","(명작, 0.918035709013503)","(판중, 0.9264555265520421)","(중국, 0.9497545084767582)","(기분, 0.9489436174167705)","(소리, 0.9524145195239514)"
2,"(중요, 0.9585038058768457)","(존경, 0.9297265394487264)","(군대, 0.9422405397511691)","(세상, 0.9598600541905957)","(발전, 0.9454924936410921)","(대작, 0.9136149774469572)","(극장, 0.9217996429641179)","(침몰, 0.9464510790766976)","(내내, 0.9401427558169381)","(집중, 0.9446176596828074)"
3,"(불편, 0.9532235068417232)","(아무, 0.9199527665875391)","(누구, 0.9418869373390635)","(모두, 0.95902505772226)","(정서, 0.9375461010932905)","(역시, 0.9124036723126655)","(그때, 0.906345958178537)","(애니, 0.9430714703027187)","(내요, 0.9384771734315531)","(바보, 0.9290489010436395)"
4,"(표현, 0.943016920601728)","(감독, 0.9167867434497288)","(걱정, 0.9363391267773004)","(자기, 0.951232981734255)","(일본, 0.9368479536397721)","(필요, 0.9114872048029306)","(여비, 0.89987773225913)","(미국, 0.9368479536397719)","(결론, 0.9333561010682919)","(장난, 0.9212729590839004)"
